# Building a Model from Scratch

```{admonition} Mildred Dresselhaus
:class: tip
People said you’re crazy... But if you think you’re right, stick to it. And we were right.
```

<iframe class="speakerdeck-iframe" frameborder="0" src="https://speakerdeck.com/player/31699a6d5e6c47f1be25e6dd16af566b" title="Machine Learning for Materials (Lecture 7)" allowfullscreen="true" style="border: 0px; background-clip: padding-box; background-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border-radius: 6px; box-shadow: rgba(0, 0, 0, 0.2) 0px 5px 40px; width: 100%; height: auto; aspect-ratio: 560 / 420;" data-ratio="1.3333333333333333"></iframe>

[Lecture slides](https://speakerdeck.com/aronwalsh/mlformaterials-lecture7-build)

## 🦾 Crystal hardness revisited

We first tackled the [bulk modulus](https://en.wikipedia.org/wiki/Bulk_modulus) of inorganic crystals in Lecture 3. However our model development was not thorough. 

Let's revisit this problem using some new tricks we have picked up. We will follow the same initial steps, making use of `matminer` (https://matminer.readthedocs.io) to access the materials dataset and featurise the data.

In [ ]:
# Installation of libraries
!pip install matminer=="0.9.0" --quiet
!pip install pymatgen=="2023.09.25" --quiet
!pip install xgboost --quiet

In [ ]:
# Import of modules
import numpy as np  # Numerical operations
from numpy import ComplexWarning  # Warning for complex numbers in NumPy
from pymatgen.core import Structure  # Materials analysis for crystal structures
from monty.serialization import loadfn  # Load serialised data
import pandas as pd  # Data manipulation with DataFrames
import matminer  # Materials informatics
from matminer.datasets.dataset_retrieval import load_dataset  # Load materials datasets
import warnings  # Warning control
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Statistical visualisation
import pprint  # Pretty print data structures
plt.style.use('ggplot')  # Set Matplotlib style to 'ggplot'
warnings.filterwarnings("ignore", category=ComplexWarning)  # Ignore ComplexWarning

# To make the model run faster
teaching_mode = True

<details>
<summary>Colab error solution</summary>
If running the import module cell fails with an "AttributeError", click `Runtime` -> `Restart Session` and then simply rerun the cell. 
</details>

## Data preparation

The steps to load and featurise the data are introduced in Notebook 3. 

In [ ]:
# Use matminer to load the dataset
df = load_dataset('matbench_log_kvrh')
print(f'The full dataset contains {df.shape[0]} entries. \n')

if teaching_mode:
  # Store the original dataframe as a copy
  full_dataset_df = df.copy()
  # Create a subset of the original dataframe for demonstration purposes
  df = df[:1500]
  print(f'For teaching purposes we will only work with {df.shape[0]} entries from the dataframe to make the model training and testing faster. \n')

print('The dataframe is shown below:')
df.head(10)

In [ ]:
# Plot a histogram of values
fig, ax = plt.subplots(figsize=(5, 4))
ax.hist(df['log10(K_VRH)'])
ax.set_xlabel(r'$log_{10}K_{VRH}$ [$log_{10}GPa$]' )
ax.set_ylabel('Counts')
plt.show()

In [ ]:
# Use matminer to featurise the dataset
from matminer.featurizers.composition.composite import ElementProperty
from matminer.featurizers.structure.order import DensityFeatures

# Add a composition column to df using the composition property of the Structure class and a lambda function
df['composition'] = df.structure.apply(lambda x: x.composition )

# Create the ElementProperty featuriser
el_prop_featuriser = ElementProperty.from_preset(preset_name='magpie')

# By default multiprocessing is enabled, however, this can slow performance, so we disable it
el_prop_featuriser.set_n_jobs(1)

# Featurise the dataframe using the ElementProperty featuriser
df = el_prop_featuriser.featurize_dataframe(df, col_id='composition')

# Add structure features
density_featuriser = DensityFeatures()
density_featuriser.set_n_jobs(1)
df=density_featuriser.fit_featurize_dataframe(df, col_id='structure')

# Print the shape of the DataFrame
print(df.shape)
df.head()

Let's understand the feature space a little better.

In [ ]:
# Extract the feature columns (excluding the first three)
feature_columns = df.columns[3:]

# Create a unique colour for each feature
colors = [plt.cm.jet(i / float(len(feature_columns))) for i in range(len(feature_columns))]

# Plot the distribution of feature values with different colours
plt.figure(figsize=(5, 4))
for i, column in enumerate(feature_columns):
    df[column].plot(kind='hist', bins=20, alpha=0.5, color=colors[i], label=column)

plt.title('Distribution of Feature Values')
plt.show()

Some dimensions have very different ranges, so we can standardise these. `MinMaxScaler` is a data scaling technique to transform numerical features within the range [0, 1]. It linearly scales data, preserving relationships between values, making it suitable for algorithms sensitive to feature magnitudes.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaled_df = df.copy()

# Step 1: Standardise the feature columns
scaler = MinMaxScaler()
scaled_df[feature_columns] = scaler.fit_transform(scaled_df[feature_columns])

# Step 2: Plot the standardised feature distributions
plt.figure(figsize=(5, 4))
for column in feature_columns:
    scaled_df[column].plot(kind='hist', bins=20, alpha=0.5, label=column)

plt.title('Standardised Feature Value Distributions')
plt.show()

Finally, let's prepare the data for model training. We need to split the dataset into the target variable `log10(K_VRH)` and the input features. For the input features, we must remove any non-numerical data to avoid getting errors later.

In [ ]:
# Define the features we want 
features_to_drop = ['structure','composition','log10(K_VRH)']
feature_cols = [col for col in list(df.columns) if col not in features_to_drop]

# Get an array of the features
X = df[feature_cols].values
scaled_X = scaled_df[feature_cols].values

# Get an array of the target variable
y = df['log10(K_VRH)'].values

print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')

## Model choice


We are dealing with a supervised regression problem, so should choose a suitable model. We can start by rebuilding a random forest. Are you curious if the feature scaling has an effect? I am.

In [ ]:
# Random forest - original features
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# Define the model
rf = RandomForestRegressor(n_estimators=100,criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42)

# Fit the model
rf.fit(X,y)

# Wrap the lines of code for later sections
def make_prediction_plot(X, y, model, label):
    y_pred = model.predict(X)  # Calculate predictions here
    fig, ax = plt.subplots(figsize=(5, 4))
    ax.scatter(y, y_pred, c=y, cmap='viridis')
    ax.plot(y, y, 'r-')
    ax.set_xlabel(f'{label} True')
    ax.set_ylabel(f'{label} Predicted')
    plt.show()
    return y_pred  # Return y_pred 

# Performance
y_pred = make_prediction_plot(X, y, rf, 'log10(K_VRH)')  

print(f'The training MAE = {metrics.mean_absolute_error(y,y_pred):.3f} log10GPa')
print(f'The training RMSE = {np.sqrt(metrics.mean_squared_error(y,y_pred)):.3f} log10GPa')
print(f'The training r^2 = {rf.score(X,y):.3f}')

In [ ]:
# Random forest - scaled features

# Define the model
rf2 = RandomForestRegressor(n_estimators=100,criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42)

# Fit the model
rf2.fit(scaled_X, y)

# Performance
y_pred = make_prediction_plot(scaled_X, y, rf2, 'log10(K_VRH)')  
print(f'The training MAE = {metrics.mean_absolute_error(y, y_pred):.3f} log10GPa')
print(f'The training RMSE = {np.sqrt(metrics.mean_squared_error(y, y_pred)):.3f} log10GPa')
print(f'The training r^2 = {rf2.score(scaled_X, y):.3f}')

We can see that Random Forest is not sensitive to feature scaling. Recall that this model works by averaging over multiple decision trees, and the decision boundaries are determined by feature thresholds, not their absolute values. 

We have time to try one more model. Let's go with the popular [XGBoost](https://xgboost.readthedocs.io). Like Random Forest, it is an ensemble learning method, but it uses a gradient-boosting framework and often achieves higher predictive accuracy by optimising for both bias and variance in the model.

In [ ]:
# XGBoost model

# Define the model
import xgboost as xgb
xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=3, random_state=42)

# Fit the model
xgb_model.fit(scaled_X, y)

# Performance
y_pred = make_prediction_plot(scaled_X, y, xgb_model, 'log10(K_VRH)') 
print(f'The training MAE = {metrics.mean_absolute_error(y, y_pred):.3f} log10GPa')
print(f'The training RMSE = {np.sqrt(metrics.mean_squared_error(y, y_pred)):.3f} log10GPa')
print(f'The training r^2 = {xgb_model.score(scaled_X, y):.3f}')

XGBoost seems to do a decent job, but we haven't performed proper training and testing yet. These models may be overfitting and unable to make predictions. On to the next stage!

## Testing and training

### Train-test split

We are ready to build a real model now. We will separate the training data from the unseen test set used to assess model performance.

In [ ]:
from slearn.model_selection import train_test_split

# Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=42)

# Print the sizes of the arrays
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

<details>
<summary> Code hint </summary>
The library is "sklearn"!
</details>

### Cross-validation 

Using the 80% training set, we can train a model by making use of cross-validation in an attempt to avoid overfitting. Note that this step may take several minutes as 10 models are being trained (i.e. 5-fold cross-validation x 2 models).

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Define the models
xgb_model = XGBRegressor(n_estimators=100, max_depth=3, random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validation for XGBoost
xgb_cv_scores = -cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
xgb_rmse = np.sqrt(xgb_cv_scores)

# Perform cross-validation for Random Forest
rf_cv_scores = -cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rf_rmse = np.sqrt(rf_cv_scores)

# Compare the results
print("XGBoost Cross-Validation Results")
print(f"  Mean RMSE: {xgb_rmse.mean():.3f}")
print(f"  Standard Deviation of RMSE: {xgb_rmse.std():.3f}")

print("\nRandom Forest Cross-Validation Results")
print(f"  Mean RMSE: {rf_rmse.mean():.3f}")
print(f"  Standard Deviation of RMSE: {rf_rmse.std():.3f}")

### Hyperparamater optimisation

XGBoost is narrowly in the lead! So far, we have not adjusted the models themselves. It is possible to improve performance by tuning the hyperparameters. Manually tuning would be laborious. We can use `GridSearchCV` to automate the search. 

Note that this step will be even more computationally expensive as we are performing cross-validation as a function of model hyperparameters for two separate models. You can see how computational cost quickly escalates and this is where powerful GPUs can become essential for machine learning! 

In [ ]:
# Warning - this cell will take 2-3 minutes to run, even with a limited search space
from sklearn.model_selection import GridSearchCV

# Hyperparameter Grid for XGBoost
xgb_param_grid = {
    'n_estimators': [200],
    'max_depth': [10],
    'learning_rate': [0.1, 0.2]
}

xgb_grid_search = GridSearchCV(XGBRegressor(random_state=42), xgb_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)

# Best hyperparameters for XGBoost
best_xgb_params = xgb_grid_search.best_params_
best_xgb_model = xgb_grid_search.best_estimator_

# Hyperparameter Grid for Random Forest
rf_param_grid = {
    'n_estimators': [200],
    'max_depth': [10],
    'min_samples_split': [2, 4]
}

rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_grid_search.fit(X_train, y_train)

# Best hyperparameters for Random Forest
best_rf_params = rf_grid_search.best_params_
best_rf_model = rf_grid_search.best_estimator_

# Evaluate the best models
xgb_cv_scores = -cross_val_score(best_xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
xgb_rmse = np.sqrt(xgb_cv_scores)

rf_cv_scores = -cross_val_score(best_rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rf_rmse = np.sqrt(rf_cv_scores)

# Compare the results of the best models
print("Best XGBoost Model Hyperparameters:", best_xgb_params)
print("Best XGBoost Cross-Validation Results")
print(f"  Mean RMSE: {xgb_rmse.mean():.3f}")
print(f"  Standard Deviation of RMSE: {xgb_rmse.std():.3f}")

print("\nBest Random Forest Model Hyperparameters:", best_rf_params)
print("Best Random Forest Cross-Validation Results")
print(f"  Mean RMSE: {rf_rmse.mean():.3f}")
print(f"  Standard Deviation of RMSE: {rf_rmse.std():.3f}")

Was it worth the effort? Well we got a small improvement in the RMSE for both models.

### Model assessment

Now that we have our best trained models, let's see how they perform on unseen test data.

In [ ]:
from sklearn.metrics import mean_squared_error

# Test the best XGBoost model
xgb_test_preds = best_xgb_model.predict(X_test)
xgb_test_rmse = np.sqrt(mean_squared_error(y_test, xgb_test_preds))

# Test the best Random Forest model
rf_test_preds = best_rf_model.predict(X_test)
rf_test_rmse = np.sqrt(mean_squared_error(y_test, rf_test_preds))

# Print test results
print("XGBoost test results:")
print(f"RMSE: {xgb_test_rmse:.3f}")

print("\nRandom Forest test results:")
print(f"RMSE: {rf_test_rmse:.3f}")

# Create a scatter plot with both models in different colors
plt.figure(figsize=(5, 4))
plt.scatter(y_test, xgb_test_preds, c='blue', label='XGBoost', alpha=0.5)
plt.scatter(y_test, rf_test_preds, c='green', label='Random Forest', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'k--', lw=2)
plt.xlabel("Actual values")
plt.ylabel("Predicted values")
plt.title("Test set performance")
plt.legend()
plt.show()

### Model speed

The speed of a model may be important, e.g. a use case involving millions of predictions. Several factors can influence the computational performance, including the dataset size, model complexity, and hardware. We can perform a simple comparison of our two models using `time`.

In [ ]:
import time

# Measure the training time for XGBoost
start_time = time.time()
xgb_model.fit(X_train, y_train)
xgb_training_time = time.time() - start_time

# Measure the training time for Random Forest
start_time = time.time()
rf_model.fit(X_train, y_train)
rf_training_time = time.time() - start_time

# Measure the prediction time for XGBoost
start_time = time.time()
xgb_test_preds = xgb_model.predict(X_test)
xgb_prediction_time = time.time() - start_time

# Measure the prediction time for Random Forest
start_time = time.time()
rf_test_preds = rf_model.predict(X_test)
rf_prediction_time = time.time() - start_time

print(f"XGBoost training time: {xgb_training_time:.3f} seconds")
print(f"Random Forest training time: {rf_training_time:.3f} seconds")
print(f"XGBoost prediction time: {xgb_prediction_time:.3f} seconds")
print(f"Random Forest prediction time: {rf_prediction_time:.3f} seconds")

## 🚨 Exercise 7: Best of the best


```{admonition} Coding exercises
:class: note
The exercises are designed to apply what you have learned with room for creativity. It is fine to discuss solutions with your classmates, but the actual code should not be directly copied.

The completed notebooks are to be submitted at the end of class, but you can revist later, experiment with the code, and follow the further reading suggestions.
```


### Your details

In [ ]:
import numpy as np

# Insert your values
Name = "No Name" # Replace with your name
CID = 123446 # Replace with your College ID (as a numeric value with no leading 0s)

# Set a random seed using the CID value
CID = int(CID)
np.random.seed(CID)

# Print the message
print("This is the work of " + Name + " [CID: " + str(CID) + "]")

### Tasks

Selecting the most appropriate ML model for a given purpose is important for achieving predictive performance. Your job is to assess additional models for the hardness regression task: 

1.  Train a [k-nearest neighbour regression](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) model and assess its performance on the test set compared to XGBoost. Some starter code is given below.

*Self-study (optional)*  

2.  Extend to a [support vector regression](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) model or other relevant regression models that you find.

```python
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# k-Nearest Neighbors Regression
knn_model = KNeighborsRegressor(n_neighbors=6)  # You can also try different n
knn_model.fit(X_train, y_train)  # Train the KNN model on the training set

# Support Vector Regression
svr_model = SVR(kernel='linear') # You could also try 'rbf' or 'poly'
svr_model.fit(X_train, y_train)  # Train the SVR model on the training set
```

<details>
<summary> Task hint </summary>
You can perform cross-validation following the same procedure as the random forest model in the notebook
</details>

```{admonition} Submission
:class: note
When your notebook is complete, click on the download icon on the top right, select `.pdf`, save the file and upload it to MyDepartment. If you are using Google Colab, you have to print to pdf.
```

In [ ]:
#Code block 




In [ ]:
#Comment block 




In [ ]:
#Code block 




In [ ]:
#Comment block 




## 🌊 Dive deeper

* _Level 1:_ Tackle Chapter 14 on Tree-Based Learners in [Machine Learning Refined](https://github.com/jermwatt/machine_learning_refined#what-is-new-in-the-second-edition). 

* _Level 2:_ Explore the XGBoost [tutorials](https://xgboost.readthedocs.io/en/stable/tutorials/model.html), e.g. predicting multiple properties with multi-output regression. 

* _Level 3:_ Find the best model (subject to time constraints) with [Automatminer](https://hackingmaterials.lbl.gov/automatminer) based on [TPOT](https://epistasislab.github.io/tpot).